# Capstone Project

### Week Three: Exploring and clustering the neighborhoods in Toronto

# Question 1

### 1. Import the Libraries

In [1]:
# Import pandas and numpy
import pandas as pd
import numpy as np
print('Pandas and numpy imported successfully')

Pandas and numpy imported successfully


In [2]:
# Lets install lxml required for the next step
!conda install -c anaconda lxml --yes
print(" Extra libraries installed")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    libxslt-1.1.33             |       h7d1a2b0_0         577 KB  anaconda
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.2 MB

The following NEW packages will be INSTALLED:

  libxslt            anaconda/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               anaconda/linux-64::lxml-4.5.0-py36hefd8a0e_0

The following packages wi

### 2. Import the table

In [78]:
# lets use the pandas get_html() to view the tables in wikipedia site
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Assign the first table to a pandas Dataframe
tdf = tables[0]
tdf.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 3. Start the data cleaning processes

In [79]:
# Let us see how many Boroughs are not assigned
tdf['Borough'].value_counts()

# 77 are not assigned
# Let us remove them

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [80]:
# Let us drop the rows with no assigned Boroughs
# 1. Let us get the index of rows with a Not assigned Borough
my_index = tdf[tdf['Borough'] == "Not assigned"].index
my_index

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [81]:
# Let us drop rows with corresponding index above
tdf.drop(my_index, inplace=True)

In [82]:
tdf.shape

(103, 3)

In [83]:
# Let us group by Borough and then combine neighbourhood values of rows with similar Boroughs
#tdf.reset_index(drop=True)
tdf = tdf.groupby('Borough').agg({'Postal Code':'first','Neighborhood':', '.join}).reset_index()
tdf

,Borough,Postal Code,Neighborhood
0,Central Toronto,M4N,"Lawrence Park, Roselawn, Davisville North, For..."
1,Downtown Toronto,M5A,"Regent Park, Harbourfront, Queen's Park, Ontar..."
2,East Toronto,M4E,"The Beaches, The Danforth West, Riverdale, Ind..."
3,East York,M4B,"Parkview Hill, Woodbine Gardens, Woodbine Heig..."
4,Etobicoke,M9A,"Islington Avenue, West Deane Park, Princess Ga..."
5,Mississauga,M7R,Canada Post Gateway Processing Centre
6,North York,M3A,"Parkwoods, Victoria Village, Lawrence Manor, L..."
7,Scarborough,M1B,"Malvern, Rouge, Rouge Hill, Port Union, Highla..."
8,West Toronto,M6H,"Dufferin, Dovercourt Village, Little Portugal,..."
9,York,M6C,"Humewood-Cedarvale, Caledonia-Fairbanks, Del R..."


In [100]:
tdf['Neighborhood']

0    Lawrence Park, Roselawn, Davisville North, For...
1    Regent Park, Harbourfront, Queen's Park, Ontar...
2    The Beaches, The Danforth West, Riverdale, Ind...
3    Parkview Hill, Woodbine Gardens, Woodbine Heig...
4    Islington Avenue, West Deane Park, Princess Ga...
5                Canada Post Gateway Processing Centre
6    Parkwoods, Victoria Village, Lawrence Manor, L...
7    Malvern, Rouge, Rouge Hill, Port Union, Highla...
8    Dufferin, Dovercourt Village, Little Portugal,...
9    Humewood-Cedarvale, Caledonia-Fairbanks, Del R...
Name: Neighborhood, dtype: object

### Get the shape of the dataframe

In [86]:
tdf.shape

(10, 3)

# Question 2

In [90]:
# Install geocoder
!pip install geocoder
print('geocoder installed')

geocoder installed


In [110]:
import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#    g = geocoder.google('Mountain View, CA')
#    lat_lng_coords = g.latlng

In [125]:
g = geocoder.google('Mountain View, CA')
lat_lng_coords = g.latlng
print(g.latlng)

None


NameError: name 'printlat' is not defined